# Spark MLlib
## Spark MLlib简介
### 什么是机器学习

机器学习是一门人工智能科学，该领域的主要研究对象是人工智能。机器学习利用数据或以往的经验，以此优化计算机程序的性能标准  
机器学习强调三个关键词：算法、经验、性能

### 基于大数据的机器学习

- 传统的机器学习算法，由于技术和单机存储的限制，只能在少量数据上使用，依赖数据抽样
- 大数据技术的出现，可以支持在全量数据上进行机器学习
- 机器学习算法涉及大量迭代计算
- 基于磁盘的MapReduce不适合进行大量迭代计算
- 基于内存的Spark比较适合进行大量迭代计算

### Spark机器学习库MLlib

- Spark提供了一个基于海量数据的机器学习库，它提供了常用机器学习算法的分布式实现
- 开发者只需要有Spark基础并且了解机器学习算法的原理，以及方法相关参数的含义，就可以轻松的调用相应的API来实现基于海量数据的机器学习过程
- pyspark的即席查询也是一个关键。算法工程师可以编写代码边看结果
- 需要注意的是，MLlib中只包含能够在集群上运行良好的并行算法，这一点很重要
- 一些较新的研究得出的算法因为使用于集群，也被包含在MLlib中，例如分布式随机森林算法、最小交替二乘法。
- 如果是小规模数据集上训练各机器学习模型，可以是用单节点的机器学习算法库（Weka）
- MLlib由一些通用的机器学习算法和工具组成，包括分类、回归、聚类、协同过滤、降维等,同时还包括底层的优化原语和高层的流水线（Pipeline）API，具体如下：
    - 算法工具：常用的学习算法，如分类、回归、聚类和协同过滤
    - 特征化工具：特征提取、转化、降维和选择工具
    - 流水线(Pipeline)：用于构建、评估和调整机器学习工作流的工具
    - 持久性：保存和加载算法、模型和管道
    - 实用工具：线性代数、统计、数据处理等工具
    
    
    
MLlib目前支持4中常见的机器学习问题：分类、回归、聚类和协同过滤
    
    
  ||离散数据|连续数据|  
  |--|--|--| 
  |监督学习|Classification、LogisticRegression(with Elastic-Net)、SVM、DecisionTree、RandomForest、GBT、NaiveBayes、MultilayerPerceptron、OneVsRest| Regression、LogisticRegression(with Elastic-Net)、DecsionTree、RandomFores、GBT、AFTSurvivalRegression、IsotonicRegression|
  |无监督学习|Clustering、KMeans、GuassianMixture、LDA、PowerIterationClustering、BisectingKMeans| Dimensionality Reduction、 matrix factorization、PCA、SVD、ALS、WLS|
    


## 机器学习流水线
### 机器学习流水线概念
- DataFrame：使用Spark SQL中的DataFrame作为数据集，它可以容纳各种数据类型。较之RDD，DataFrame包含了schema信息，更类似传统数据库的二维表格。
- 它被ML Pipeline用来存储源数据。例如，DataFrame中的列可以是存储的文本、特征向量、真实标签和预测的标签等

- Transformer：转换器，是一种可以将一个DataFrame转换为另一个DataFrame的算法。比如一个模型就是一个Transformer。它可以把一个不包含预测标签等测试数据集DataFrame打上标签，转换成另一个包含预测标签的DataFrame
- Estimator：估计器或评估器，它是学习算法或在训练数据上的训练方法的概念抽象。在Pipeline里通常是被用来操作DataFrame数据并生成一个Transformer。从技术上讲，Estimator实现了一个方法fit()，它接受一个DataFrame并产生一个转换器。比如一个随机森林算法就是一个Estimator，它可以调用fit()，通过训练特征数据而得到一个随机深林算法
- Parameter：Parameter被用来设置Transformer或者Estimator的参数。现在，所有转换器和评估器可共享用于指定参数的公共API。ParamMap是一组（参数，值）对
- PipeLine：流水线或者管道。流水线将多个工作流阶段（转换器和评估器）连接在一起，形成机器学习的工作流，并获得结果输出。


### 流水线工作流程
要构建一个Pipeline流水线，首先需要定义Pipeline中的各个流水线阶段PipelineStage(包括转换器和评估器），比如指标提取和转换模型训练等。有了这些处理特定问题的转换器和评估器，就可以按照具体的处理逻辑有序地组织PipelineStages并创建一个Pipeline
```python
pipeline = Pipeline(stages=[stage1, stage2, stage3]
```

然后就可以把训练数据集作为输入参数，调用Pipeline实例的fit方法来开始以流的方式来处理源训练数据。这个调用会返回一个PipelineModel类实例，进而被用来预测测试数据的标签

- 流水线的各个阶段按顺序运行，输入的DataFrame在它通过每个阶段时被转换

值得注意的是，流水线本身也可以看作是一个评估器，在流水线的fit()方法运行之后，它产生一个PipelineModel，它是一个Transformer。这个管道模型将在测试数据的时候使用

### 构建一个机器学习流水线
以逻辑斯蒂回归为例，构建一个典型的机器学习过程

**任务描述**  
查找出所有包含“spark”的句子，将包含“spark”的句子的标签设为1，没有“spark”的句子的标签设为0
- 需要使用SparkSession对象


In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('WordCount').getOrCreate()

- 引入要包含的包并构建数据集

In [2]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

#Prepare training documents from a list of (id, text, label) tuples
training = spark.createDataFrame([
    (0, 'a b c d e spark', 1.0),
    (1, 'b d', 0.0),
    (2, 'spark f g h', 1.0),
    (3, 'hadoop mapreduce', 0.0),
], ['id', 'text', 'label'])

/home/spark/miniconda3/envs/bigdata/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


- 定义Pipeline中的各个流水线阶段PipelineStage，包括转换器和评估器，具体地，包含tokenizer，hashing TF和lr

In [3]:
tokenizer = Tokenizer(inputCol='text', outputCol='words')
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol='features')
lr = LogisticRegression(maxIter=10, regParam=0.001)

- 按照具体的处理逻辑有序地组织PipelineStages，并创建一个Pipeline

In [4]:
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

现在构建的Pipeline本质上是一个Estimator，在它的fit()方法运行之后，它将产生一个PipelineModel，它是一个Transformer

In [5]:
model = pipeline.fit(training)

可以看到，model的类型是一个PipelineModel，这个流水线模型将在测试数据的时候使用


- 构建测试数据

In [6]:
test = spark.createDataFrame([
    (4, 'spark i j k'),
    (5, 'I m n'),
    (6, 'spark hadoop spark'),
    (7, 'apache hadoop')
], ['id', 'text'])

- 调用之前训练好的PipelineModel的transform()方法，让测试数据按顺序通过拟合的流水线，生成预测结果

In [8]:
prediction = model.transform(test)
selected = prediction.select('id', 'text', 'probability', 'prediction')
for row in selected.collect():
    rid, text, prob, prediction = row
    print("(%d, %s) --> prob=%s, prediction=%f" % (rid, text, str(prob), prediction))

(4, spark i j k) --> prob=[0.1596407738787475,0.8403592261212525], prediction=1.000000
(5, I m n) --> prob=[0.8378325685476744,0.16216743145232562], prediction=0.000000
(6, spark hadoop spark) --> prob=[0.06926633132976037,0.9307336686702395], prediction=1.000000
(7, apache hadoop) --> prob=[0.9821575333444218,0.01784246665557808], prediction=0.000000


## 特征提取和转换
### 特征提取

- “词频-逆向文件频率”(TF-IDF)是一种在文本挖掘中广泛使用的特征向量化方法，它可以体现一个文档中词语在语料库中的重要程度
- 词语由t表示，文档由d表示，语料库由D表示。词频TF(t, d)是词语t在文档d中出现的次数。文件频率DF(t, D)是包含词语的文档的个数
- TF-IDF就是在数值化文档信息，衡量词语能够提供多少信息以区分文档。其定义如下:
$$
        IDF(t,D) = log \frac {|D| + 1}{DF(t,D)+1}\

        TFIDF(t,d,D) = TF(t,d)·IDF(t,D)
$$

在Spark ML库中，TF-IDF被分成两部分：
- TF(+hashing)
    HashingTF是一个Transformer，在文本处理中，接收词条的集合然后把这些集合转化为固定长度的特征向量。这个算法在哈希的同时会统计各个词条的词频。
        
- IDF
    IDF是一个Estimator，在一个数据集上应用它的fit()方法，产生一个IDFModel。该IDFModel接收特征向量（由HashingTF产生），然后计算每一个词在文档中出现的频次。IDF会减少那些在语料库中出现频率较高的词的权重。

**过程描述**
- 在下面的代码段中，我们以一组句子开始
- 首先使用分解器Tokenizer把句子划分为单个词语
- 对每一个句子（词带），使用HashingTF将句子转换为特征向量
- 最后使用IDF重新调整特征向量（这种转换通常可以提高使用文本特征的性能）
    
    

(1)导入TF-IDF所需要的包

In [9]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

(2)创建一个简单的DataFrame，每一个句子代表一个文档

In [10]:
sentenceData = spark.createDataFrame([
    (0, 'I heard about Spark and I love Spark'),
    (0, 'I wish Java could use case classes'),
    (1, 'Logistic regression models are neat')
]).toDF('label', 'sentence')

(3)得到文档集合后，即可用tokenizer对句子进行分词

In [11]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')
wordsData = tokenizer.transform(sentenceData)
wordsData.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|    0|I heard about Spa...|[i, heard, about,...|
|    0|I wish Java could...|[i, wish, java, c...|
|    1|Logistic regressi...|[logistic, regres...|
+-----+--------------------+--------------------+



(4)得到分词后的文档序列后，即可使用HashingTF的transform()方法把句子哈希成特征向量，这里设置哈希表的桶数为2000

In [12]:
hashingTF = HashingTF(inputCol='words', outputCol='rawFeatures', numFeatures=2000)
featurizedData = hashingTF.transform(wordsData)
featurizedData.select('words', 'rawFeatures').show(truncate=False)

+---------------------------------------------+---------------------------------------------------------------------+
|words                                        |rawFeatures                                                          |
+---------------------------------------------+---------------------------------------------------------------------+
|[i, heard, about, spark, and, i, love, spark]|(2000,[240,333,1105,1329,1357,1777],[1.0,1.0,2.0,2.0,1.0,1.0])       |
|[i, wish, java, could, use, case, classes]   |(2000,[213,342,489,495,1329,1809,1967],[1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|[logistic, regression, models, are, neat]    |(2000,[286,695,1138,1193,1604],[1.0,1.0,1.0,1.0,1.0])                |
+---------------------------------------------+---------------------------------------------------------------------+



(5)调用IDF方法来构造特征向量的规模，生成的变量idf是一个评估器，在特征向量上应用它的fit()方法，会产生一个IDFModel（名称为idfModel）

In [13]:
idf = IDF(inputCol='rawFeatures', outputCol='features')
idfModel = idf.fit(featurizedData)

(6)调用IDFModel的transform()方法，可以得到每一个单词对应的TF-IDF度量值

In [14]:
rescaledData = idfModel.transform(featurizedData)
rescaledData.select('features', 'label').show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                                                                                       |label|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|(2000,[240,333,1105,1329,1357,1777],[0.6931471805599453,0.6931471805599453,1.3862943611198906,0.5753641449035617,0.6931471805599453,0.6931471805599453])                       |0    |
|(2000,[213,342,489,495,1329,1809,1967],[0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453])|0    |
|(2000,[286,695,1138,1193,1604],[0.6931471805599453,0.6931471805599453,0.6931471

### 特征转换：标签和和索引的转换

- 在机器学习处理过程中，为了方便相关算法的实现，经常需要把标签数据（一般是字符串）转化成整数索引，或是在计算结束后将整数索引还原为相应的标签
- Spark ML包中提供了几个相关的转换器，例如，StringIndexer、IndexToString、OneHotEncoder、VectorIndexer，它们提供了十分方便的特征转换功能，这些转换器都位于org.apache.spark.ml.feature包下
- 值得注意的是，用于特征转换的转换器和其他的机器学习算法一样，也属于ML Pipeline模型的一部分，可以用来构成机器学习流水线，以StringIndexer为例，其存储着进行标签数值化过程的相关超参数，是一个Estimator，对其调用fit()方法即可生成相应的模型StringIndexModel类，很显然，它存储于用于DataFrame进行相关处理的参数，是一个Transformer

- StringIndexer
    - StringIndexer转换器可以把一列类别型的特征（或标签）进行编码，使其数值化，索引的范围从0开始，该过程可以使得相应的特征索引化，使得某些无法接受类别特征的算法可以使用，并提高诸如决策树等机器学习算法等效率
    - 索引构建的顺序为标签的频率，优先编码频率较大的标签，所以出现频率最高的标签为0号
    

(1)首先，引入所需要使用的类

In [6]:
from pyspark.ml.feature import StringIndexer

(2)其次，构建1个DataFrame，设置StringIndexer的输入列和输出列的名字

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').getOrCreate()
df = spark.createDataFrame([(0, 'a'), (1, 'b'), (2, 'c'), (3, 'a'), (4, 'a'), (5, 'c')],
                          ['id', 'category'])
indexer = StringIndexer(inputCol='category', outputCol='categoryIndex')

(3)然后，通过fit()方法进行模型训练，用训练出的模型对原数据集进行处理，并通过indexed.show()进行展示

In [9]:
model = indexer.fit(df)
indexed = model.transform(df)
indexed.show()

+---+--------+-------------+
| id|category|categoryIndex|
+---+--------+-------------+
|  0|       a|          0.0|
|  1|       b|          2.0|
|  2|       c|          1.0|
|  3|       a|          0.0|
|  4|       a|          0.0|
|  5|       c|          1.0|
+---+--------+-------------+



- IndexToString
    - 与StringIndexer相对应，IndexToString的作用是把标签索引的一列重新映射回原有的字符型标签
    - 其主要使用场景一般都是和StringIndexer配合，先用StringIndexer将标签转化成标签索引，进行模型训练，然后在预测标签的时候再把标签索引转化成原有的字符标签
    

In [10]:
from pyspark.ml.feature import IndexToString, StringIndexer
toString = IndexToString(inputCol='categoryIndex', outputCol='originalCategory')
indexString = toString.transform(indexed)
indexString.select('id', 'originalCategory').show()

+---+----------------+
| id|originalCategory|
+---+----------------+
|  0|               a|
|  1|               b|
|  2|               c|
|  3|               a|
|  4|               a|
|  5|               c|
+---+----------------+



- VectorIndexer
    - 之前介绍的StringIndexer是针对单个类别型特征进行转换，倘若所有特征都已经被组织在一个向量中，又想对其中某些单个分量进行处理时，Spark ML提供了VectorIndexer类来解决向量数据集中的类别性特征转换
    - 通过为其提供maxCategories超参数，它可以自动识别哪些特征是类别型的，并且将原始值转换为类别索引。它基于不同特征值的数量来识别哪些特征需要被类别化，那些取值可能性最多不超过maxCategories的特征需要会被认为是类别型的

In [11]:
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.linalg import Vector, Vectors

df = spark.createDataFrame([
    (Vectors.dense(-1.0, 1.0, 1.0),),
    (Vectors.dense(-1.0, 3.0, 1.0),),
    (Vectors.dense(0.0, 5.0, 1.0),)], ['features'])

然后，构建VectorIndexer转换器，设置输入和输出列，并进行模型训练

In [12]:
indexer = VectorIndexer(inputCol='features', outputCol='indexed', maxCategories=2)
indexerModel = indexer.fit(df)

接下来，通过VectorIndexerModel的categoryMaps成员来获得被转换的特征及其映射，这里可以看到，共有两个特征被转换，分别是0号和2号

In [14]:
categoricalFeatures = indexerModel.categoryMaps.keys()
print('Choose '+ str(len(categoricalFeatures)) + 'categorical features: ' + str(categoricalFeatures))

Choose 2categorical features: KeysView({0: {0.0: 0, -1.0: 1}, 2: {1.0: 0}})


最后，把模型应用于原有的数据，并打印结果

In [15]:
indexed = indexerModel.transform(df)
indexed.show()

+--------------+-------------+
|      features|      indexed|
+--------------+-------------+
|[-1.0,1.0,1.0]|[1.0,1.0,0.0]|
|[-1.0,3.0,1.0]|[1.0,3.0,0.0]|
| [0.0,5.0,1.0]|[0.0,5.0,0.0]|
+--------------+-------------+



## 分类与回归
### 逻辑斯蒂回归分类器
逻辑斯蒂回归(logistic regression)是统计学习中的经典分类方法，属于对数线性模型。logistic回归的因变量可以是二分类也可以是多分类。

任务描述：以iris数据集（iris）为例进行分析  
iris以鸢尾花的特征作为数据来源，数据集包含150个数据集，分为3类，每类50个数据，每个数据包含4个属性，是在数据挖掘、数据分类中非常常用的测试集、训练集。为了便于理解，这里主要用后两个属性（花瓣的长度和宽度）来进行分类

In [16]:
!wget http://dblab.xmu.edu.cn/blog/wp-content/uploads/2017/03/iris.txt

--2021-12-13 19:30:32--  http://dblab.xmu.edu.cn/blog/wp-content/uploads/2017/03/iris.txt
Resolving dblab.xmu.edu.cn (dblab.xmu.edu.cn)... 210.34.0.35, 2001:da8:e800::35
Connecting to dblab.xmu.edu.cn (dblab.xmu.edu.cn)|210.34.0.35|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘iris.txt’

    [ <=>                                   ] 4,698       --.-K/s   in 0.001s  

2021-12-13 19:30:32 (5.81 MB/s) - ‘iris.txt’ saved [4698]



首先我们先取其中的后两类数据，用二项逻辑斯蒂回归进行二分类分析

第1步：导入本地向量Vector和Vectors，导入所需要的类

In [3]:
from pyspark.ml.linalg import Vector, Vectors
from pyspark.sql import Row, functions
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, HashingTF, Tokenizer
from pyspark.ml.classification import LogisticRegression, LogisticRegressionModel, \
                    BinaryLogisticRegressionSummary

from pyspark.sql import SparkSession

第2步：我们制定一个函数，来返回一个指定的数据然后读取文本文件，第一个map把每行的数据用“,”隔开，比如在我们的数据集中，每行被分成了5部分，前4部分是鸢尾花的4个特征，最后一部分是鸢尾花的分类；我们在这里把特征存储在Vector中，创建一个Iris模式的RDD，然后转化成dataframe；最后调用show()来查看部分数据

In [4]:
def f(x):
    rel = {}
    rel['features'] = Vectors.dense(float(x[0]), float(x[1]), float(x[2]), float(x[3]))
    rel['label'] = str(x[4])
    return rel

spark = SparkSession.builder.master('local').getOrCreate()
data = spark.sparkContext.textFile('file:///home/spark/code/SparkProgramming-PySpark/studying/iris.txt')\
        .map(lambda line: line.split(',')).map(lambda p: Row(**f(p))).toDF()
data.show()

+-----------------+-----------+
|         features|      label|
+-----------------+-----------+
|[5.1,3.5,1.4,0.2]|Iris-setosa|
|[4.9,3.0,1.4,0.2]|Iris-setosa|
|[4.7,3.2,1.3,0.2]|Iris-setosa|
|[4.6,3.1,1.5,0.2]|Iris-setosa|
|[5.0,3.6,1.4,0.2]|Iris-setosa|
|[5.4,3.9,1.7,0.4]|Iris-setosa|
|[4.6,3.4,1.4,0.3]|Iris-setosa|
|[5.0,3.4,1.5,0.2]|Iris-setosa|
|[4.4,2.9,1.4,0.2]|Iris-setosa|
|[4.9,3.1,1.5,0.1]|Iris-setosa|
|[5.4,3.7,1.5,0.2]|Iris-setosa|
|[4.8,3.4,1.6,0.2]|Iris-setosa|
|[4.8,3.0,1.4,0.1]|Iris-setosa|
|[4.3,3.0,1.1,0.1]|Iris-setosa|
|[5.8,4.0,1.2,0.2]|Iris-setosa|
|[5.7,4.4,1.5,0.4]|Iris-setosa|
|[5.4,3.9,1.3,0.4]|Iris-setosa|
|[5.1,3.5,1.4,0.3]|Iris-setosa|
|[5.7,3.8,1.7,0.3]|Iris-setosa|
|[5.1,3.8,1.5,0.3]|Iris-setosa|
+-----------------+-----------+
only showing top 20 rows



第3步：分别获取标签列和特征列，进行索引并进行重命名

In [5]:
labelIndexer = StringIndexer().setInputCol('label').setOutputCol('indexedLabel').fit(data)
featureIndexer = VectorIndexer().setInputCol('features').setOutputCol('indexedFeatures').fit(data)

第4步：设置LogisticRegression算法的参数。这里设置了循环次数为100次，规范化项为0.3等，具体可以设置的参数，可以通过explainParams()来获取，还能看到程序已经设置的参数的结果

In [6]:
lr = LogisticRegression().setLabelCol('indexedLabel').setFeaturesCol('indexedFeatures')\
            .setMaxIter(100).setRegParam(0.3).setElasticNetParam(0.8)
print('LogisticRegression parameters: \n' + lr.explainParams())

LogisticRegression parameters: 
aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0, current: 0.8)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: indexedFeatures)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: indexedLabel)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on int

第5步：设置一个IndexToString的转换器，把预测的类别重新转化成字符型的。构建一个机器学习流水线，设置各个阶段.

In [7]:
labelConverter = IndexToString().setInputCol('prediction').setOutputCol('predictionLabel')\
            .setLabels(labelIndexer.labels)
LrPipeline = Pipeline().setStages([labelIndexer, featureIndexer, lr, labelConverter])

第6步：把数据集随机分成训练集和测试集，其中训练集占70%。Pipeline本质就是一个评估器，当Pipeline调用fit()的时候就产生了一个PipelineModel就可以调用transform()来预测，生成一个新的DataFrame，即利用训练得到的模型对测试集进行验证

In [9]:
trainingData, testData = data.randomSplit([0.7, 0.3])
lrPipelineModel = LrPipeline.fit(trainingData)
lrPredictions= lrPipelineModel.transform(testData)

第7步：输出预测结果，其中，select选择要输出的列，collect获取所有行的数据

In [11]:
preRel = lrPredictions.select(
    'predictionLabel', 'label', 'features', 'probability'
).collect()

In [14]:
for item in preRel:
    print(str(item['label']) + ',' + str(item['features']) + '-->prob=' + str(item['probability']) +\
                                         ', predictionLabel' + str(item['predictionLabel']))

Iris-setosa,[4.3,3.0,1.1,0.1]-->prob=[0.5422572686691972,0.26976850602033237,0.1879742253104703], predictionLabelIris-setosa
Iris-setosa,[4.4,2.9,1.4,0.2]-->prob=[0.5138979821868874,0.2825238997163162,0.2035781180967964], predictionLabelIris-setosa
Iris-setosa,[4.4,3.2,1.3,0.2]-->prob=[0.5200663364012367,0.2789388343933832,0.20099482920538006], predictionLabelIris-setosa
Iris-setosa,[4.6,3.2,1.4,0.2]-->prob=[0.5138979821868874,0.2825238997163162,0.2035781180967964], predictionLabelIris-setosa
Iris-setosa,[4.7,3.2,1.3,0.2]-->prob=[0.5200663364012367,0.2789388343933832,0.20099482920538006], predictionLabelIris-setosa
Iris-setosa,[4.8,3.0,1.4,0.3]-->prob=[0.5039029296970303,0.2842715877055052,0.21182548259746442], predictionLabelIris-setosa
Iris-setosa,[4.8,3.4,1.9,0.2]-->prob=[0.4830302989816677,0.3004642865379552,0.21650541448037705], predictionLabelIris-setosa
Iris-versicolor,[4.9,2.4,3.3,1.0]-->prob=[0.32268501260340204,0.3487034070187629,0.32861158037783506], predictionLabelIris-vers

第8步：对训练的模型进行评估。创建一个Multiclass Classification Evaluator实例，用setter方法把预测分类的列名和真实的列名进行设置，然后计算预测准确率

In [15]:
evaluator = MulticlassClassificationEvaluator().setLabelCol('indexedLabel').setPredictionCol('prediction')
lrAccuracy = evaluator.evaluate(lrPredictions)
lrAccuracy

0.6885304659498208

第9步：通过model来获取训练得到的逻辑斯蒂模型。LrPipelineModel是一个PipelineModel，因此可以通过它的stages来获取模型

In [17]:
lrModel = lrPipelineModel.stages[2]
print('Coefficients: \n' + str(lrModel.coefficientMatrix) + '\nIntercept: ' + str(lrModel.interceptVector)\
             + '\nnumClasses: ' + str(lrModel.numClasses) + '\nnumFeatures: ' + str(lrModel.numFeatures))

Coefficients: 
3 X 4 CSRMatrix
(0,2) -0.247
(0,3) -0.2581
(1,3) 0.3355
Intercept: [0.7954077176700615,-0.2003011786498851,-0.5951065390201763]
numClasses: 3
numFeatures: 4


### 决策树分类器
决策树(decision tree)是一种基本的分类与回归方法，这里主要介绍用于分类的决策树。决策树呈树形结构，其中每个内部节点表示一个属性上的测试，每个分支代表一个测试输出，每个叶节点代表一种类别。学习时利用训练数据，根据损失函数最小化的原则建立决策树模型；预测时，对新的数据。利用决策树模型进行分类。  

决策树学习通常包括3个步骤：特征选择、决策树的生成和决策树的剪枝

继续使用鸢尾花数据集，数据集包含150个数据集，分为3类，每类50个数据，每个数据包含4个属性，是在挖掘数据、数据分类中非常常用的测试集、训练集

In [19]:
#第1步：导入需要用的包
from pyspark.ml.classification import DecisionTreeClassificationModel
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.linalg import Vector, Vectors
from pyspark.sql import Row, SparkSession
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer

spark = SparkSession.builder.master('local').getOrCreate()

In [22]:
#第2步：读取文本文件，第一个map把每行的数据用','隔开，比如我们的数据集中，每行被分成了5部分，前4部分是鸢尾花的4个特征，最后一部分
#是鸢尾花的分类；把特征存储在Vector中，创建一个iris模式的RDD，然后转化为DataFrame
def f(x):
    rel = {}
    rel['features'] = Vectors.dense(float(x[0]), float(x[1]), float(x[2]), float(x[3]))
    rel['label'] = str(x[4])
    return rel

data = spark.sparkContext.textFile('file:///home/spark/code/SparkProgramming-PySpark/studying/iris.txt')\
        .map(lambda line: line.split(',')).map(lambda p: Row(**f(p))).toDF()

In [23]:
#第3步：进步处理特征和标签，把数据集随机分成训练集和测试集
labelIndexer = StringIndexer().setInputCol('label').setOutputCol('indexedLabel').fit(data)
featureIndexer = VectorIndexer().setInputCol('features')\
            .setOutputCol('indexedFeatures').setMaxCategories(4).fit(data)
labelConverter = IndexToString().setInputCol('prediction')\
            .setOutputCol('predictedLabel').setLabels(labelIndexer.labels)
trainingData, testData = data.randomSplit([0.7, 0.3])

In [24]:
#第4步：创建决策树模型，设置决策树参数
dtClassifier = DecisionTreeClassifier().setLabelCol('indexedLabel').setFeaturesCol('indexedFeatures')

In [25]:
#第5步：构建机器学习流水线(Pipeline)，在训练数据集上调用fit()进行模拟训练，并在测试数据集调用transform()方法进行预测
dtPipeline = Pipeline().setStages([labelIndexer, featureIndexer, dtClassifier, labelConverter])
dtPipelineModel = dtPipeline.fit(trainingData)
dtPredictions = dtPipelineModel.transform(testData)
dtPredictions.select('predictedLabel', 'label', 'features').show(20)

+---------------+---------------+-----------------+
| predictedLabel|          label|         features|
+---------------+---------------+-----------------+
|    Iris-setosa|    Iris-setosa|[4.4,2.9,1.4,0.2]|
|    Iris-setosa|    Iris-setosa|[4.4,3.2,1.3,0.2]|
|    Iris-setosa|    Iris-setosa|[4.6,3.4,1.4,0.3]|
|    Iris-setosa|    Iris-setosa|[4.8,3.0,1.4,0.1]|
|    Iris-setosa|    Iris-setosa|[4.8,3.0,1.4,0.3]|
|    Iris-setosa|    Iris-setosa|[4.9,3.0,1.4,0.2]|
|    Iris-setosa|    Iris-setosa|[4.9,3.1,1.5,0.1]|
|Iris-versicolor|Iris-versicolor|[5.0,2.0,3.5,1.0]|
|Iris-versicolor|Iris-versicolor|[5.0,2.3,3.3,1.0]|
|    Iris-setosa|    Iris-setosa|[5.0,3.5,1.6,0.6]|
|    Iris-setosa|    Iris-setosa|[5.1,3.4,1.5,0.2]|
|    Iris-setosa|    Iris-setosa|[5.1,3.5,1.4,0.3]|
|    Iris-setosa|    Iris-setosa|[5.1,3.8,1.6,0.2]|
|    Iris-setosa|    Iris-setosa|[5.1,3.8,1.9,0.4]|
|    Iris-setosa|    Iris-setosa|[5.4,3.4,1.7,0.2]|
|Iris-versicolor|Iris-versicolor|[5.5,2.3,4.0,1.3]|
|Iris-versic

In [26]:
evaluator = MulticlassClassificationEvaluator().setLabelCol('indexedLabel').setPredictionCol('prediction')
dtAccuracy = evaluator.evaluate(dtPredictions)
dtAccuracy

0.9534883720930232

In [27]:
#第6步：查看决策树模型结构
treeModelClassifier = dtPipelineModel.stages[2]
print('Learned classification tree model: \n' + str(treeModelClassifier.toDebugString))

Learned classification tree model: 
DecisionTreeClassificationModel (uid=DecisionTreeClassifier_17b9f5a2b286) of depth 5 with 15 nodes
  If (feature 2 <= 2.45)
   Predict: 0.0
  Else (feature 2 > 2.45)
   If (feature 3 <= 1.65)
    If (feature 2 <= 4.95)
     Predict: 1.0
    Else (feature 2 > 4.95)
     If (feature 0 <= 6.05)
      If (feature 1 <= 2.25)
       Predict: 2.0
      Else (feature 1 > 2.25)
       Predict: 1.0
     Else (feature 0 > 6.05)
      Predict: 2.0
   Else (feature 3 > 1.65)
    If (feature 2 <= 4.85)
     If (feature 1 <= 2.8499999999999996)
      Predict: 2.0
     Else (feature 1 > 2.8499999999999996)
      Predict: 1.0
    Else (feature 2 > 4.85)
     Predict: 2.0

